In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import json
import re
import random
from matplotlib import pyplot as plt
#{"None":0,"zh_char":1,"zh_str":2,"en":3,"zh_en":4}

/home/mark/Desktop/py3.7/cv/paddleOCR/lib/python3.7/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
dataset_path = "/home/mark/Desktop/py3.7/cv/datasets/ai_cup_f/"

In [48]:
df = pd.read_csv(dataset_path+"lsvt_max_bound.csv",encoding="utf8")
df.head()

,imgs,pt1,pt2,pt3,pt4,pt5,pt6,pt7,pt8,labels
0,gt_1234,45,272,215,273,212,296,45,290,一站式购物中心
1,gt_1234,33,325,204,328,202,363,31,357,母婴百汇
2,gt_1234,25,359,71,364,70,385,25,382,TEL:
3,gt_1234,72,362,197,365,197,388,72,384,8168886
4,gt_1234,649,317,665,317,662,331,648,331,方


In [49]:
df = df.fillna("###")

### 去除###的資料

In [50]:
non_empty = df[df["labels"]!="###"]
len(non_empty)

241855

In [51]:
l = []
for elem in non_empty.labels:
    l.append(len(elem))
print(len(l))

241855


In [52]:
#觀察labels分佈
str_count = {}
for elem in list(set(l)):
    str_count[str(elem)] = l.count(elem)
str_count

{'1': 26658,
 '2': 40072,
 '3': 30063,
 '4': 43857,
 '5': 18299,
 '6': 15249,
 '7': 11109,
 '8': 8302,
 '9': 6224,
 '10': 5504,
 '11': 6994,
 '12': 3961,
 '13': 3242,
 '14': 4743,
 '15': 2352,
 '16': 2208,
 '17': 1492,
 '18': 1117,
 '19': 991,
 '20': 832,
 '21': 743,
 '22': 672,
 '23': 729,
 '24': 547,
 '25': 527,
 '26': 818,
 '27': 488,
 '28': 446,
 '29': 323,
 '30': 291,
 '31': 273,
 '32': 256,
 '33': 222,
 '34': 219,
 '35': 188,
 '36': 181,
 '37': 142,
 '38': 151,
 '39': 172,
 '40': 105,
 '41': 104,
 '42': 85,
 '43': 88,
 '44': 100,
 '45': 73,
 '46': 69,
 '47': 51,
 '48': 65,
 '49': 58,
 '50': 39,
 '51': 41,
 '52': 38,
 '53': 30,
 '54': 33,
 '55': 10,
 '56': 25,
 '57': 18,
 '58': 25,
 '59': 13,
 '60': 15,
 '61': 5,
 '62': 16,
 '63': 11,
 '64': 4,
 '65': 10,
 '66': 14,
 '67': 4,
 '68': 6,
 '69': 5,
 '70': 4,
 '71': 9,
 '72': 3,
 '73': 5,
 '74': 2,
 '75': 2,
 '76': 6,
 '80': 2,
 '83': 1,
 '85': 1,
 '90': 1,
 '92': 1,
 '94': 1}

In [53]:
#set random to train split
x = (non_empty.imgs.unique())
len(x)

29990

### 設定要不要分train,test

In [59]:
# train = non_empty
# test=[]

In [54]:
# from sklearn.model_selection import train_test_split

# train, test = train_test_split(x, test_size=0.2,random_state=69)
# test[:10]

array(['gt_2638', 'gt_18842', 'gt_7600', 'gt_9029', 'gt_14265', 'gt_6721',
       'gt_20529', 'gt_9783', 'gt_25244', 'gt_17244'], dtype=object)

In [57]:
#test = test.tolist()

In [60]:
out_f = open(dataset_path+"/train_det_max/train_det_labels1.txt","a",encoding="utf8")#a持續寫下去 w覆寫
out_f1 = open(dataset_path+"/train_det_max/test_det_labels1.txt","a",encoding="utf8")
lst=[]
#save_lst=[]
previous_label = None
for index, row in non_empty.iterrows():
    pts = [[row['pt1'],row['pt2']],[row['pt3'],row['pt4']],[row['pt5'],row['pt6']],[row['pt7'],row['pt8']]]
    out_dic = {"transcription":row["labels"],"points":pts}
    if previous_label:
        if previous_label != row["imgs"]:
            if previous_label in test:#確認資料屬於test or train 
                out_f1.write("./lsvt/"+previous_label+".jpg\t"+json.dumps(lst,ensure_ascii=False)+'\n')#存前一張圖的檔
                previous_label= row["imgs"] 
                lst = [] #empty list
            else:
                out_f.write("./lsvt/"+previous_label+".jpg\t"+json.dumps(lst,ensure_ascii=False)+'\n')#存前一張圖的檔
                previous_label= row["imgs"] 
                lst = [] #empty list
    else:
        previous_label= row["imgs"]
    lst.append(out_dic)
out_f.close()
out_f1.close()